In [1]:
import requests
import pandas as pd
import psycopg2
import json
from redshift_utils import get_api_key,cargar_en_redshift,conn_redshift

# Weather API

Consiste en llamadas a ´http://api.weatherapi.com´ para traer temperatura minimas y temperaturas maximas en grados celsius.

Para eso cree esta notebook donde hago un loop en un rango de fechas e inserto los datos en la BD y luego un script que corre con la fecha actualizada de cada dìa.

In [3]:
key=get_api_key(".env/api_key.txt")
base_url = "http://api.weatherapi.com/v1/history.json"

date="2023-05-01"
url = f"{base_url}?key={key}&q=Buenos Aires&dt={date}"
response = requests.get(url)
data = response.json()
data

{'location': {'name': 'Buenos Aires',
  'region': 'Distrito Federal',
  'country': 'Argentina',
  'lat': -34.59,
  'lon': -58.67,
  'tz_id': 'America/Argentina/Buenos_Aires',
  'localtime_epoch': 1684185288,
  'localtime': '2023-05-15 18:14'},
 'forecast': {'forecastday': [{'date': '2023-05-01',
    'date_epoch': 1682899200,
    'day': {'maxtemp_c': 20.5,
     'maxtemp_f': 68.9,
     'mintemp_c': 10.9,
     'mintemp_f': 51.6,
     'avgtemp_c': 15.0,
     'avgtemp_f': 59.0,
     'maxwind_mph': 8.3,
     'maxwind_kph': 13.3,
     'totalprecip_mm': 1.5,
     'totalprecip_in': 0.06,
     'avgvis_km': 9.9,
     'avgvis_miles': 6.0,
     'avghumidity': 57.0,
     'condition': {'text': 'Sunny',
      'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
      'code': 1000},
     'uv': 6.0},
    'astro': {'sunrise': '07:31 AM',
     'sunset': '06:13 PM',
     'moonrise': '04:17 PM',
     'moonset': '03:16 AM',
     'moon_phase': 'Waxing Gibbous',
     'moon_illumination': '78'},
    'hour'

In [3]:
autor = []
autor_nacimiento = []
autor_muerte = []
libro = []
genero = []
download_count = []

for i in range(0,32):
    autor.append(data["results"][i]["authors"][0]["name"])
    autor_nacimiento.append(data["results"][i]["authors"][0]["birth_year"])
    autor_muerte.append(data["results"][i]["authors"][0]["death_year"])
    libro.append(data["results"][i]["title"])
    #genero.append(data["results"][i]["bookshelves"])
    download_count.append(data["results"][i]["download_count"])

df = pd.DataFrame({"autor":autor,"año_nacimiento":autor_nacimiento
                ,"año_muerte":autor_muerte,"titulo":libro,"download_count":download_count})

df.sort_values(by="download_count", ascending=False, inplace=True)


In [4]:
df.head()

,autor,año_nacimiento,año_muerte,titulo,download_count
0,"Vonnegut, Kurt",1922,2007,2 B R 0 2 B,3341
1,"Hunter, Evan",1926,2005,Reaching for the moon,1545
2,"Harrison, Harry",1925,2012,The stainless steel rat,1382
3,"Marlowe, Stephen",1928,2008,Fugue,1005
4,"Vonnegut, Kurt",1922,2007,The Big Trip Up Yonder,775


In [5]:
conn_db_coder = conn_redshift(".env/pwd_coder.txt")
cargar_en_redshift(conn=conn_db_coder, table_name="books",dataframe=df)

Connected to Redshift successfully!
Proceso terminado
